In [ ]:
import networkx as nx
import random
import json
import plotly.graph_objects as go
import numpy as np
#from google.colab import drive
import os

# Function to generate graphs with vertices and edges based on the provided packages
def generate_graph(numNodes, V_R, V_B, adjList):
    G = nx.Graph()

    # Generate vertices with R and B values
    for i in range(numNodes):
        R = V_R[i]
        B = V_B[i]
        G.add_node(i, R=R, B=B)

    # Generate edges with L values
    # Iterates over all nodes to generate edges with values ​​of L
    # Each pair (neighbor, L) in the sublist adjList[i]
    # neighbor represents the node connected to node i, and L is the edge weight (latency) or associated value
    for i in range(numNodes):
     for neighbor, L in adjList[i]:
        G.add_edge(i, neighbor, L=L)

    print(f"graph: {G}")

    return G

# Function to find the shortest path (minimum spanning tree) using Kruskal's algorithm
def kruskal_minimum_path(G):
    mst = nx.minimum_spanning_tree(G, weight='L', algorithm='kruskal')
    return mst

# Function that generates the intermediate graph including the Kruskal path
def generate_intermediate_graph(G, mst):
    # Copy the complete graph G
    G_intermediate = G.copy()

    return G_intermediate

# Function to generate random 3D coordinates for graph nodes
def generate_3D_coordinates(G):
    # Número de nós no grafo
    num_nodes = len(G.nodes)

    # Generates random coordinates for each node, within a 100x100x100 cube
    pos = {}
    for node in G.nodes:
        x = np.random.uniform(0, 100)
        y = np.random.uniform(0, 100)
        z = np.random.uniform(0, 100)
        pos[node] = (x, y, z)  # Armazena as coordenadas 3D para o nó

    return pos

# Function to convert the graph to random 3D coordinates
# These values ​​are random coordinates generated for each node in 3D space.
# The function generate_3D_coordinates calculates the coordinates and stores them in the
# dictionary pos, where each node receives a tuple containing the coordinates; x, y and z.
def display_3D_graph(G, pos, title="3D Graph"):
    edge_x = []
    edge_y = []
    edge_z = []
    edge_text = []  # List to store latency texts

    # Processing the edges
    for edge in G.edges(data=True):
        x0, y0, z0 = pos[edge[0]]
        x1, y1, z1 = pos[edge[1]]
        edge_x.extend([x0, x1, None])
        edge_y.extend([y0, y1, None])
        edge_z.extend([z0, z1, None])

        # Calculating the midpoint of an edge
        mx, my, mz = (x0 + x1) / 2, (y0 + y1) / 2, (z0 + z1) / 2
        edge_text.append(f"L: {edge[2]['L']:.2f}")  # Adicionando a latência

    edge_trace = go.Scatter3d(
        x=edge_x, y=edge_y, z=edge_z,
        line=dict(width=2, color='gray'),
        hoverinfo='none',
        mode='lines',
        name='Edges')

    # Processing the vertices
    node_x = []
    node_y = []
    node_z = []
    node_text = []

    for node in G.nodes(data=True):
        x, y, z = pos[node[0]]
        node_x.append(x)
        node_y.append(y)
        node_z.append(z)
        node_text.append(f"<b>N</b>: {node[0]}<br><b>R</b>: {node[1]['R']}<br><b>B</b>: {node[1]['B']}")

    node_trace = go.Scatter3d(
        x=node_x, y=node_y, z=node_z,
        mode='markers+text',
        textposition="middle center",  # To display attributes in the middle of nodes (vertices)
        hoverinfo='text',
        marker=dict(
            showscale=False,
            color='lightgreen',
            size=45,   # Size of the circle representing the nodes (vertices)
            line_width=1.0),
        text=node_text,
        name='Active Nodes')

    # Display latencies at the midpoint of edges
    edge_text_trace = go.Scatter3d(
        x=[(pos[edge[0]][0] + pos[edge[1]][0]) / 2 for edge in G.edges()],
        y=[(pos[edge[0]][1] + pos[edge[1]][1]) / 2 for edge in G.edges()],
        z=[(pos[edge[0]][2] + pos[edge[1]][2]) / 2 for edge in G.edges()],
        mode='text',
        text=[f"<b>L</b>: {G.edges[edge]['L']}" for edge in G.edges()],
        hoverinfo='text',
        textposition="middle center",
        name='Latencies'
    )

    # Set the chart title
    title = title

    # Create the layout with the title centered and include the captions in a list of notes
    annotations = [
        dict(
            text="<b>N</b> = Node<br><b>R</b> = Resource<br><b>B</b> = Bandwidth<br><b>L</b> = Latency",  # Create the layout with the title centered and include the captions in a list of notes
            showarrow=False,  # No arrow
            xref="paper", yref="paper",  # Reference to the total area of ​​the graph (not the 3D scene)
            x=0.9, y=1.0,  # Adjust as needed
            xanchor='left',  # Align left
            yanchor='top',  # Positions the caption at the top of the area
            align='left',  # Ensures left alignment
            font=dict(size=14, color="black")
        )
    ]

    layout = go.Layout(
        title=dict(
            text=title,
            x=0.5,
            xanchor='center',
            font=dict(size=20)
        ),
        showlegend=True,  # Enable subtitles
        scene=dict(
            xaxis=dict(showbackground=False, visible=False),
            yaxis=dict(showbackground=False, visible=False),
            zaxis=dict(showbackground=False, visible=False)
        ),
        margin=dict(l=0, r=0, b=100, t=100),  # Adjusting the bottom and top margins
        width=1600,
        height=1200,
        annotations=annotations  # Assign the entire notes list to the layout
    )

    # Create the 3D graph figure
    fig = go.Figure(data=[edge_trace, node_trace, edge_text_trace], layout=layout)

    # Display the chart
    fig.show()

#------------------------------ End functions ----------------------------------

# Main Function
def main():

    # --- Locate and read the input file to visualize the IoT network ---
    #drive.mount('/content/drive')

    # Set the default path where the .json files are located
    default_in_f = r'C:\Users\jonat\OneDrive\Área de Trabalho\Jonatas\input_Nviwer'

    # Prompt the user for the input directory path or use the default value
    input_folder = input(f"Enter an input directory path or press ENTER for default (ex: {default_in_f}): ") or default_in_f

    # List all .json files in the input directory
    json_files = [f for f in os.listdir(input_folder) if f.endswith('.json')]

    # Process each .json file and generate the graph visualization
    for json_file in json_files:
      # Full path of the input file
      path_input = os.path.join(input_folder, json_file)
      # Load input data from a JSON file
      with open(path_input, 'r') as f:
        data = json.load(f)

    #---------------------------------------------------------------------------
    #-------------- Values ​​assigned to the IoT network -------------------------
    # EN =  Network Edge, RN = Resource Network
    # Stores the number of edge layer nodes
    edge_nodes = np.array(data['edge_nodes'])
    # --------------------------------------------------------------------------
    # Initial values ​​of vectors R and B
    N_R = np.array(data['V_R'])
    N_B = np.array(data['V_B'])
    #----------------- No. of IoT network nodes (Variables)---------------------
    # Collecting the number of IoT network nodes
    numNodes = len(N_R)
    # Initializing adjacency lists as a list
    adjList = data['adjList']
    #---------------------------------------------------------------------------

    # Generate the graph based on the given parameters
    G = generate_graph(numNodes, N_R, N_B, adjList)

    # Calculate the minimum spanning tree (Kruskal)
    mst = kruskal_minimum_path(G)

    # Generate the random intermediate graph + Kruskal
    G_intermediate = generate_intermediate_graph(G, mst)

    # Generate 3D coordinates for graphs
    pos = generate_3D_coordinates(G)

    # Display the complete graph
    #display_3D_graph(G, pos, title="Complete 3D Graph")
    #print(f"G_complete: {G}")

    # Display the intermediate graph
    display_3D_graph(G_intermediate, pos, title="IoT Network")
    print(f"G_intermediate: {G_intermediate}")

    # Display the minimum spanning tree (Kruskal)
    display_3D_graph(mst, pos, title="Minimum Spanning Tree (Kruskal) 3D")
    print(f"G_mst: {mst}")

    # Initial values ​​of vectors: V_Busy and V_Inactive
    V_Busy = np.array(data['V_Busy'])  # Reads values ​​from the input file
    V_Inactive = np.array(data['V_Inactive'])  # Reads values ​​from the input file
    #---------------------------------------------------------------------------
    # Running random.randint numJob times and storing the values ​​in a list
    #------------------------- Job parameter values ----------------------------
    jr = np.array(data['jr'])
    jb = np.array(data['jb'])
    jl = np.array(data['jl'])
    jo = np.array(data['jo'])

    # Collecting the number of jobs to be executed on the IoT network
    numJob = len(jr)  # Nº of jobs
    #---------------------------------------------------------------------------

if __name__ == "__main__":
    main()
